# `11-chatbot.ipynb`

# Build Chatbot
- https://python.langchain.com/docs/tutorials/chatbot/

In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [4]:
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    HumanMessage(content='Hi, I am Bob'),
    AIMessage(content='Hello Bob. How can I help you?'),
    HumanMessage(content='Say my name.')
]

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

res = llm.invoke(messages)

res.pretty_print()

================================== Ai Message ==================================

Hello, Bob!


In [5]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph

# Graph Builder
builder = StateGraph(state_schema=MessagesState)

# Node
def simple_node(state: MessagesState):
    res = llm.invoke(state['messages'])
    return {'messages': res}

builder.add_node('simple_node', simple_node)

# Edge (Node 끼리 연결)
builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

# Memory (대화내역 기록)
memory = MemorySaver()

# Graph (그래프 생성)
graph = builder.compile(checkpointer=memory)

In [6]:
# 설정(conf, config, configuration -> 설정)
config = {'configurable': {'thread_id': 'abc123'}} # 채팅방 아이디

graph.invoke({'messages': messages}, config=config)

{'messages': [HumanMessage(content='Hi, I am Bob', additional_kwargs={}, response_metadata={}, id='ff726c65-01a0-4090-9714-95a56ee73057'),
  AIMessage(content='Hello Bob. How can I help you?', additional_kwargs={}, response_metadata={}, id='f7aa3ca9-6e59-4028-bb86-3bd2587fe5a4'),
  HumanMessage(content='Say my name.', additional_kwargs={}, response_metadata={}, id='feec7b44-f46d-4dbb-ab93-bc5cc27631e8'),
  AIMessage(content='Hello, Bob!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 33, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CDKSxOwO6JtAW2IiaYsKZGTWfOM5E', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--07ac0c

In [8]:
import uuid

u_id = uuid.uuid1()
print(u_id)

config = {'configurable': {'thread_id': '가나다123'}} # 채팅방 아이디 -> 추후에는 UUID 형식으로 생성
messages = [
    HumanMessage(content='내 이름은 말해.')
]

graph.invoke({'messages': messages}, config=config)

b46c96b4-8c4f-11f0-8a1f-a0d36593c8de


{'messages': [HumanMessage(content='내 이름은 말해.', additional_kwargs={}, response_metadata={}, id='aab0c3bf-b010-4e1a-b5c2-ccdd9c1ad5d2'),
  AIMessage(content='죄송하지만, 저는 당신의 이름을 알지 못해요. 혹시 알려주실 수 있나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 13, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_7c233bf9d1', 'id': 'chatcmpl-CDKUiExTmgr71k3ITkawFKjhToaG1', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ccd541cd-0b12-448e-8eee-6d51cbc2cb41-0', usage_metadata={'input_tokens': 13, 'output_tokens': 24, 'total_tokens': 37, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
  HumanMessage(conte

## Langgraph + `PromptTemplate`

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ('system', 'You need to think and speak like gen-z'), 
    MessagesPlaceholder(variable_name='messages')
])

# 실행 예시
for msg in prompt_template.invoke({'messages': ['hi']}).messages:
    print(msg)

content='You need to think and speak like gen-z' additional_kwargs={} response_metadata={}
content='hi' additional_kwargs={} response_metadata={}


## State 확장

In [29]:
# 내장된 MessagesState를 확장해서 사용
class MyState(MessagesState):
    # 상속받아서 이미 key 'messages'는 있음
    # messages: Annotated[list[AnyMessage], add_messages]
    lang: str

builder = StateGraph(state_schema=MyState)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐. {lang} 언어로 답해.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])

def simple_node(state: MyState):
    # prompt 추가.
    chain = prompt_template | llm  # 체인 방식
    res = chain.invoke(state)

    return {'messages': res}

builder.add_node('simple_node', simple_node)

builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [30]:
config = {'configurable': {'thread_id': 'abc1'}}
state = {
    'messages':[HumanMessage(content='안녕 나는 이상한 나라의 앨리스야')],
    'lang': 'Chinese'
}

res = graph.invoke(state, config)

for msg in res['messages']:
    msg.pretty_print()

================================ Human Message =================================

안녕 나는 이상한 나라의 앨리스야
================================== Ai Message ==================================

你好，爱丽丝！很高兴见到你。你来自奇幻的奇境，想要探索什么样的奇妙冒险呢？


## 대화 기록 관리하기

대화 내역을 관리하지 않으면, LLM의 컨텍스트 윈도우(입력 최대치)를 넘어가 버림.

In [33]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages

# trim - 정리하다
trimmer = trim_messages(
    strategy='last',        # 최신 메세지들을
    max_tokens=65,           # 최대 65 토큰까지만 허용
    token_counter=llm,      # llm 모델에 맞춰서 토큰 세고
    include_system=True,    # system 프롬포트는 포함(정리X)
    allow_partial=False,    # 메세지 중간에서 자르지는 말고
    start_on='human'        # 잘린 메세지의 첫번째는 사람 메세지가 되도록
)

In [ ]:
messages = [
    SystemMessage(content="you're a good assistant"),
    
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)  # 시스템 메세지 -> 2+2 부터 등장

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [ ]:
class MyState(MessagesState):
    # messages: ~~
    lang: str


builder = StateGraph(state_schema=MyState)

prompt_template = ChatPromptTemplate.from_messages([
    ('system', '너는 유능한 어시스턴트야. 너의 능력을 최대한 활용해서 답을 해봐. {lang} 언어로 답해.'),
    MessagesPlaceholder(variable_name='messages')  # 모든 저장된 대화 내용(최신것 포함)
])


def simple_node(state: MyState):
    # 메세지 정리 -> 프롬프트 생성 -> LLM 답변
    # print('정리 전 메시지 개수: ', len(state['messages']))
    trimmed_messages = trimmer.invoke(state['messages'])
    # print('정리 후 메시지 개수: ', len(trimmed_messages))
    
    # 체인 생성
    chain = prompt_template | llm
    
    # 정리된 메세지로 state 교체 후, 체인 실행
    state['messages'] = trimmed_messages
    res = chain.invoke(state)

    return {'messages': res}


builder.add_node('simple_node', simple_node)

builder.add_edge(START, 'simple_node')
builder.add_edge('simple_node', END)

memory = MemorySaver()

graph = builder.compile(checkpointer=memory)

In [ ]:
config = {'configurable': {'thread_id': 'abc1'}}
state = {
    'messages':[HumanMessage(content='한국 라면 추천')],
    'lang': 'Chinese'
}

for chunk, metadata in graph.stream(state, config, stream_mode='messages'):
    print(chunk.content, end='|')

정리 전 메세지 개수:  5
정리 후 메세지 개수:  1
================================ Human Message =================================

한국 라면 추천 좀
	 None
|当然|可以|！|以下|是|一些|受|欢迎|的|韩国|拉|面|推荐|：

|1|.| 辛|拉|面|（|Sh|in| Ram|yun|）|  
|  | -| 特|点|：|辣|味|十|足|，|汤|头|浓|郁|，|面|条|弹|牙|。|  
|  | -| |适|合|喜欢|辣|味|的|朋友|。

|2|.| 三|养|火|鸡|面|（|Sam|yang| Hot| Chicken| Flavor| R|amen|）|  
|  | -| 特|点|：|极|辣|，|风|靡|全球|的|“|火|鸡|面|”。|  
|  | -| 建|议|逐|步|尝|试|，|适|合|喜欢|挑战|辣|味|的|食|客|。

|3|.| |皱|眉|面|（|J|j|apag|etti|）|  
|  | -| 特|点|：|非|辣|口|味|，|偏|向|酱|料|，|口|感|丰富|。|  
|  | -| |适|合|不|喜欢|辣|的|朋友|。

|4|.| |牧|羊|人|拉|面|（|Ne|og|uri|）|  
|  | -| 特|点|：|海|鲜|味|浓|郁|，|汤|底|偏|咸|鲜|，|面|条|宽|厚|。|  
|  | -| |适|合|喜欢|海|鲜|风|味|的|食|客|。

|5|.| |方便|面|（|B|uld|ak| Bok|ke|um| My|un|）|  
|  | -| 特|点|：|辣|味|酱|炒|面|，|辣|度|较|高|，|口|感|丰富|。|  
|  | -| |适|合|喜欢|辣|味|和|炒|面的|朋友|。

|如果|你|喜欢|辣|味|，|推荐|辛|拉|面|和|火|鸡|面|；|如果|偏|好|清|淡|或|海|鲜|口|味|，可以|试|试|皱|眉|面|或|牧|羊|人|拉|面|。|希望|你|能|找到|喜欢|的|韩国|拉|面|！||